## BARREL Spectroscopy Crib


### Intro 
In this guide we will explore the spectroscopy functions provided by the BARREL Spectroscopy functions in pySPEDAS. In general, these functions can be accessed as `pyspedas.barrel.spec.<function_name>`. These functions have been ported from the BARREL module of the IDL SPEDAS package. The examples in this notebook are derived from `barrel_spectroscopy_crib.pro`.

### Setup

>**_NOTE:_** The `%matplotlib ipympl` line will enable interactive plots. This is not required, but will allow you to use your cursor to see exact times and values. The `ipympl` package may need to be installed separately from pyspedas.

In [ ]:
%matplotlib ipympl
import pyspedas, pytplot

### Example 1: bkg time intervals; exponential spectrum, slow spectra

> **_Original IDL syntax:_**\
barrel_spectroscopy,spectest1,'2013-01-17/00:50:00',9.,'1G',/slow,numbkg=2,fitrange=[80.,2500.],saveme='spectest1.sav'

> 


In this example we will download 9 hours of slow spectra data from payload 1G starting at `2013-01-17/00:50:00` using exponential spectrum.

First, the we define the define a `trange` variable and load the CDF files. We get get both the SSPC and FSPC CDF files:

In [ ]:
trange = ["2013-01-17/00:50:00", "2013-01-17/09:50:00"]
pyspedas.barrel.sspc(trange, "1G")
pyspedas.barrel.fspc(trange, "1G")

Using the `tplot_names()` function lists the variable names that have been downloaded (we are looking for `brl1G_SSPC`).

In [ ]:
pyspedas.tplot_names()

Now that the data have been downloaded, we can plot the SSPC data: 

In [ ]:
pytplot.tplot('brl1G_SSPC')
pytplot.tplot('brl1G_FSPC1')

Using the plots, identify the event and background time periods:

In [ ]:
event_period=("2013-01-17/01:54:00", "2013-01-17/03:24:00")
background_periods=[("2013-01-17/17:25:00", "2013-01-17/20:35:00"), ("2013-01-18/09:35:00", "2013-01-18/12:04:00")]

Next we can initalize our spectroscopy data object `specstruct`:

In [ ]:
specstruct = pyspedas.barrel.spec.barrel_sp_make(tplot_var="brl1G_SSPC", event_periods=event_period, background_periods=background_periods)

Once the `specstruct` has been created, we can build the spectra from the CDF files that have been loaded (this will also download rate counter and ephemeris file automatically):

In [ ]:
pyspedas.barrel.spec.barrel_sp_collect_spectra(specstruct)

The `specstrut` object contains a source spectrum and background spectrum which can be plotted. We will extract the raw data from the SSPC CDF file to get the energy levels, then create new tplot variables for the source and background spectra and plot them.

In [ ]:
ts, raw_cnts, energy_levels = pytplot.get_data('brl1G_SSPC')

pytplot.store_data("brl1G_Event_Spec", data={'x':energy_levels, 'y':specstruct["src_spec"]})
pytplot.options("brl1G_Event_Spec", opt_dict={"name": "Event Spectrum", "ytitle": "cnts/keV/sec"})
pytplot.tplot("brl1G_Event_Spec")

pytplot.store_data("brl1G_Bkg_Spec", data={'x':energy_levels, 'y':specstruct["bkg_spec"]})
pytplot.options("brl1G_Bkg_Spec", opt_dict={"name": "Background Spectrum", "ytitle": "cnts/keV/sec"})
pytplot.tplot("brl1G_Bkg_Spec")